In [57]:
import pandas as pd
import numpy as np
import psycopg2 as pg
import locale
from datetime import date
from sqlalchemy import create_engine
from sqlalchemy.schema import CreateSchema


In [58]:
locale.setlocale(locale.LC_TIME, 'pt_BR')
csv_1 = '/caminho_do_arquivo.csv'
csv_2 = '/caminho_do_arquivo.csv'
schema_Name = 'grupos_vulneraveis'
table_Name = 'todos'

In [59]:
def main() -> None:

    '''Main function that handles the pipeline flow. Steps:
        1. Calls the extract function to extract data from the CSV file and save it in a dataframe that will be used in the transform function.
        2. Calls the transform function to perform data modeling in the extracted dataframe and save the results in another dataframe that will be used in the lead function.
        3. Calls the load function to save the transformed dataframe in the database.

        Parameters:
        - None
    
        Returns:
        - None'''
    
    df_extracted = extract(csv_1, csv_2)
    df_transformed = transform(df_extracted)
    load(df_transformed, schema_Name, table_Name)

In [60]:
def extract(file_path_1: str, file_path_2: str) -> pd.DataFrame:

    '''Extract the data from two CSV files and save in one dataframe. Steps:
        1. Read the CSV files separately using pandas and save them in saparete dataframes;
        2. Join the two dataframes using pandas.

        Parameters:
        - String containing the first CSV file path;
        - String containing the second CSV file path.

        Return:
        - Joined dataframe.'''
    
    df1 = pd.read_csv(file_path_1, encoding = 'latin1', sep = ";")
    df2 = pd.read_csv(file_path_2, encoding = 'latin1', sep = ";")

    df = pd.concat([df1, df2], axis = 0, ignore_index = True)
    print("DataFrame criado com sucesso.")
    
    display(df.head(5))
    
    return df

In [61]:
def add_columns(df: pd.DataFrame) -> pd.DataFrame:

    '''Add new columns for the dataframe. Steps:
        1. Creates a new column and assigns the dataframe's update date to it.
        2. Creates a new column and assigns the extracted year from a specific date column to it using pandas.
        3. Creates a new column and assigns the extracted month from the same date column used to extract the year to it using pandas.

        Parameters:
        - Dataframe for which the columns will be added.

        Returns:
        - Dataframe with the added columns.'''

    
    df['atualizado_em'] = date.today()
    print("Coluna 'atualizado_em' adicionada ao DataFrame.")

    df['ano_fato']= pd.to_datetime(df['data_fato']).dt.year.astype(str).str.replace('.0', '')
    print("Coluna 'ano_fato' adicionada ao DataFrame.")

    df['mes_fato']= pd.to_datetime(df['data_fato']).dt.strftime('%B')
    print("Coluna 'mes_fato' adicionada ao DataFrame.")

    return df


In [62]:
def transform(df: pd.DataFrame) -> pd.DataFrame:
    '''Transform the dataframe by calling the add columns function and changing some columns types using pandas. Steps:
        1. Calls the add columns function and save in a dataframe;
        2. Uses pandas to convert the type of a date column from the dataframe from text to datetime and then to date;
        3. Uses pandas to convert the type of a hour column from the dataframe from text to datetime and then to time;
        4. Uses replace function to replace NULL values to a text;
        5. Uses pandas to convert the type of a age column from the dataframe from float to text and then remove unnecessaries character.
        
        Parameters:
        - Dataframe to be transformed.
        
        Return:
        - Dataframe transformed.'''

    df = add_columns(df)

    df['data_com'] = pd.to_datetime(df['data_com']).dt.date
    print("Coluna 'data_com' convertida para DATE.")

    df['hora_com'] = pd.to_datetime(df['hora_com'], format='%H:%M:%S').dt.time
    print("Coluna 'hora_com' convertida para TIME.")

    df = df.replace(np.nan, 'Não informado')
    print("Valores NULL do dataframe substituidos por 'Não informado'.")

    df['idade'] = df['idade'].astype(str).str.replace('.0', '')
    print("Coluna 'idade' convertida para STRING.")

    display(df.head(5))
    df.info()

    return df

In [63]:
def get_postgre_connection_parameters() -> dict:

    '''Creates a dict containing the connection parameters info.
        
        Parameters:
        - None
        
        Return:
        - The database connection or the error message.'''
        
    conn_parameters = {
        'host': 'host_do_banco',
        'database': 'nome_do_banco',
        'user': 'usuário',
        'password': 'senha'
    }

    return conn_parameters

In [64]:
def load(df: pd.DataFrame, schema_name: str, table_name: str) -> None:

    '''Load the transformed dataframe into the postgre database. Steps:
        1. Calls the postgre connection parameters function to receive the parameters;
        2. Creates a SQLAlchemy object for the conection using the parameters connection in the postgre connection function;
        3. Sets the schema and creates it if doesn't exist;
        4. Loads the transformed dataframe into the database;
        5. Closes the connection;
        6. Prints a successfully message if the ingestion is complete;
        7. Prints the error message if it's not.
        
        Parameters:
        - Dataframe to be load;
        - String with the schema name where the table will be load;
        - String containing the name the table will be called.
        
        Return:
        - None.'''

    conn_params = get_postgre_connection_parameters()

    try:
        engine = create_engine(f"postgresql+psycopg2://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}/{conn_params['database']}")
        
        with engine.connect() as connection:
            connection.execute(CreateSchema(schema_name, if_not_exists=True))
            connection.commit()

        df.to_sql(table_name, con=engine, index=False, if_exists='replace', schema=schema_name)
        print(">> Ingestão de dados concluída com sucesso <<")
        print("Conexão fechada.")

    except Exception as error:
        print(f"Erro ao fazer a ingestão de dados no banco de dados: {error}")


In [66]:
if __name__ == "__main__":

    main()

DataFrame criado com sucesso.


,controle,ano,mes,titulo,titulo_do,total_rbft,lei,conteudo,dp,cisp,...,local,sexo,data_nasc,idade,cor,escolaridade,profissao,relacao,bairro_vit,municipio_vit
0,00787401-2020,2020,Janeiro,"Lesão Corporal Provocada por Socos, Tapas e Po...",Lesão corporal dolosa,Não se aplica,Não,vítimas,038a. Braz de Pina,038a. Braz de Pina,...,Residência,feminino,1975-07-02,44.0,parda,1º Grau incompleto,Cozinheiro(a),Outra,Cordovil,Rio de janeiro
1,00987771-2020,2020,Janeiro,Homicídio Provocado por Projétil de Arma de Fogo,Homicídio doloso,Não se aplica,Não,vítimas,123a. Macaé,123a. Macaé,...,Via pública,masculino,1983-02-28,36.0,parda,1º Grau incompleto,Pintor(a),Nenhuma,Fronteira,Macae
2,00987771-2020,2020,Janeiro,Homicídio Provocado por Projétil de Arma de Fogo,Homicídio doloso,Não se aplica,Não,vítimas,123a. Macaé,123a. Macaé,...,Via pública,feminino,1970-12-25,49.0,ignorado,Ignorado,Ignorado,Nenhuma,Praia campista,Macae
3,00987771-2020,2020,Janeiro,Homicídio Provocado por Projétil de Arma de Fo...,Tentativa de homicídio,Não se aplica,Não,vítimas,123a. Macaé,123a. Macaé,...,Via pública,masculino,1999-09-10,20.0,negra,Ignorado,Ignorado,Nenhuma,Granja dos cavaleiros,Macae
4,00987771-2020,2020,Janeiro,Homicídio Provocado por Projétil de Arma de Fo...,Tentativa de homicídio,Não se aplica,Não,vítimas,123a. Macaé,123a. Macaé,...,Via pública,masculino,1996-06-19,23.0,parda,Ignorado,Ignorado,Nenhuma,Fronteira,Macae


Coluna 'atualizado_em' adicionada ao DataFrame.
Coluna 'ano_fato' adicionada ao DataFrame.
Coluna 'mes_fato' adicionada ao DataFrame.
Coluna 'data_com' convertida para DATE.
Coluna 'hora_com' convertida para TIME.
Valores NULL do dataframe substituidos por 'Não informado'.
Coluna 'idade' convertida para STRING.


,controle,ano,mes,titulo,titulo_do,total_rbft,lei,conteudo,dp,cisp,...,idade,cor,escolaridade,profissao,relacao,bairro_vit,municipio_vit,atualizado_em,ano_fato,mes_fato
0,00787401-2020,2020,Janeiro,"Lesão Corporal Provocada por Socos, Tapas e Po...",Lesão corporal dolosa,Não se aplica,Não,vítimas,038a. Braz de Pina,038a. Braz de Pina,...,44,parda,1º Grau incompleto,Cozinheiro(a),Outra,Cordovil,Rio de janeiro,2024-02-01,2020,Janeiro
1,00987771-2020,2020,Janeiro,Homicídio Provocado por Projétil de Arma de Fogo,Homicídio doloso,Não se aplica,Não,vítimas,123a. Macaé,123a. Macaé,...,36,parda,1º Grau incompleto,Pintor(a),Nenhuma,Fronteira,Macae,2024-02-01,2020,Janeiro
2,00987771-2020,2020,Janeiro,Homicídio Provocado por Projétil de Arma de Fogo,Homicídio doloso,Não se aplica,Não,vítimas,123a. Macaé,123a. Macaé,...,49,ignorado,Ignorado,Ignorado,Nenhuma,Praia campista,Macae,2024-02-01,2020,Janeiro
3,00987771-2020,2020,Janeiro,Homicídio Provocado por Projétil de Arma de Fo...,Tentativa de homicídio,Não se aplica,Não,vítimas,123a. Macaé,123a. Macaé,...,20,negra,Ignorado,Ignorado,Nenhuma,Granja dos cavaleiros,Macae,2024-02-01,2020,Janeiro
4,00987771-2020,2020,Janeiro,Homicídio Provocado por Projétil de Arma de Fo...,Tentativa de homicídio,Não se aplica,Não,vítimas,123a. Macaé,123a. Macaé,...,23,parda,Ignorado,Ignorado,Nenhuma,Fronteira,Macae,2024-02-01,2020,Janeiro


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814962 entries, 0 to 814961
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   controle        814962 non-null  object
 1   ano             814962 non-null  int64 
 2   mes             814962 non-null  object
 3   titulo          814962 non-null  object
 4   titulo_do       814962 non-null  object
 5   total_rbft      814962 non-null  object
 6   lei             814962 non-null  object
 7   conteudo        814962 non-null  object
 8   dp              814962 non-null  object
 9   cisp            814962 non-null  object
 10  data_com        814962 non-null  object
 11  data_fato       814962 non-null  object
 12  hora_com        814962 non-null  object
 13  hora_fato       814962 non-null  object
 14  bairro_fato     814962 non-null  object
 15  municipio_fato  814962 non-null  object
 16  aisp            814962 non-null  object
 17  risp            814962 non-nu